# SwellSight Real-to-Synthetic Pipeline - Setup and Installation (Enhanced)

This notebook handles environment preparation and dependency management for the SwellSight real-to-synthetic image generation pipeline with integrated utilities.

## New Features
- ✨ Integrated SwellSight utilities for better error handling and progress tracking
- 🔧 Automatic configuration management
- 📊 Memory optimization and monitoring
- 🛡️ Robust error handling with retry logic
- 📈 Progress tracking and performance feedback
- 🚀 Updated for Depth-Anything-V2 and FLUX.1-dev models

---

## 1. Import SwellSight Utilities

In [ ]:
import sys
import os
from pathlib import Path
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Add utils directory to Python path
utils_path = Path.cwd() / "utils"
if str(utils_path) not in sys.path:
    sys.path.insert(0, str(utils_path))

try:
    # Import SwellSight utilities
    from utils import (
        load_config, validate_config,
        validate_image_quality, validate_depth_map_quality,
        get_optimal_batch_size, cleanup_variables, monitor_memory,
        retry_with_backoff, handle_gpu_memory_error,
        create_progress_bar, display_stage_summary,
        save_stage_results, load_previous_results, check_dependencies
    )
    print("✅ SwellSight utilities loaded successfully")
except ImportError as e:
    print(f"⚠️ Could not import utilities: {e}")
    print("Continuing with basic functionality...")

## 2. Load Configuration and Environment Setup

In [ ]:
# Load pipeline configuration
try:
    config = load_config("config.json")
    print(f"✅ Configuration loaded: {config['pipeline']['name']} v{config['pipeline']['version']}")
    
    # Extract commonly used settings
    batch_size = config['processing']['batch_size']
    quality_threshold = config['processing']['quality_threshold']
    data_dir = Path(config['paths']['data_dir'])
    output_dir = Path(config['paths']['output_dir'])
    checkpoint_dir = Path(config['paths']['checkpoint_dir'])
    
    print(f"📁 Data directory: {data_dir}")
    print(f"📁 Output directory: {output_dir}")
    print(f"📁 Checkpoint directory: {checkpoint_dir}")
    print(f"🎯 Quality threshold: {quality_threshold}")
    
    # Display model configuration
    print(f"\n🤖 Model Configuration:")
    print(f"  Depth Model: {config['models']['depth_model']}")
    print(f"  Base Model: {config['models']['base_model']}")
    print(f"  ControlNet: {config['models']['controlnet_model']}")
    print(f"  Mixed Precision: {config['models']['mixed_precision']}")
    
except Exception as e:
    print(f"⚠️ Could not load configuration: {e}")
    print("Using default settings...")
    
    # Fallback configuration
    batch_size = "auto"
    quality_threshold = 0.7
    data_dir = Path("./data")
    output_dir = Path("./outputs")
    checkpoint_dir = Path("./checkpoints")
    
    config = {
        'pipeline': {'name': 'swellsight_pipeline', 'version': '1.0'},
        'processing': {'batch_size': batch_size, 'quality_threshold': quality_threshold},
        'models': {
            'depth_model': 'depth-anything/Depth-Anything-V2-Large',
            'base_model': 'black-forest-labs/FLUX.1-dev',
            'controlnet_model': 'Shakker-Labs/FLUX.1-dev-ControlNet-Depth',
            'mixed_precision': True
        },
        'paths': {'data_dir': str(data_dir), 'output_dir': str(output_dir), 'checkpoint_dir': str(checkpoint_dir)}
    }

## 3. Enhanced Hardware Detection and GPU Memory Optimization

In [ ]:
import torch
import platform

# Check if running in Google Colab
if 'google.colab' in sys.modules:
    from google.colab import drive
    print("🔗 Mounting Google Drive...")
    
    def mount_drive():
        drive.mount('/content/drive')
        return True
    
    try:
        # Use retry logic for drive mounting
        retry_with_backoff(mount_drive, max_retries=2)
        print("✅ Google Drive mounted successfully")
    except Exception as e:
        print(f"⚠️ Drive mounting failed: {e}")

# Enhanced hardware detection with FLUX requirements
print("\n🔍 Hardware Detection:")
print(f"Platform: {platform.system()} {platform.release()}")
print(f"Python: {sys.version.split()[0]}")

# GPU detection with FLUX memory requirements
flux_memory_requirements = {
    'minimum_gb': 8,
    'recommended_gb': 16,
    'optimal_gb': 24
}

if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    current_gpu = torch.cuda.current_device()
    gpu_name = torch.cuda.get_device_name(current_gpu)
    gpu_memory = torch.cuda.get_device_properties(current_gpu).total_memory / (1024**3)
    
    print(f"🚀 GPU Available: {gpu_name}")
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
    print(f"🔢 GPU Count: {gpu_count}")
    
    # FLUX memory assessment
    if gpu_memory >= flux_memory_requirements['optimal_gb']:
        print(f"✅ GPU memory is optimal for FLUX.1-dev ({gpu_memory:.1f} GB >= {flux_memory_requirements['optimal_gb']} GB)")
        flux_compatibility = "optimal"
    elif gpu_memory >= flux_memory_requirements['recommended_gb']:
        print(f"⚠️ GPU memory is adequate for FLUX.1-dev ({gpu_memory:.1f} GB >= {flux_memory_requirements['recommended_gb']} GB)")
        flux_compatibility = "adequate"
    elif gpu_memory >= flux_memory_requirements['minimum_gb']:
        print(f"⚠️ GPU memory is minimal for FLUX.1-dev ({gpu_memory:.1f} GB >= {flux_memory_requirements['minimum_gb']} GB)")
        print("💡 Consider using smaller batch sizes and mixed precision training")
        flux_compatibility = "minimal"
    else:
        print(f"❌ GPU memory insufficient for FLUX.1-dev ({gpu_memory:.1f} GB < {flux_memory_requirements['minimum_gb']} GB)")
        print("💡 Consider using CPU fallback or upgrading GPU")
        flux_compatibility = "insufficient"
        
    # Check for mixed precision support
    if torch.cuda.get_device_capability(current_gpu)[0] >= 7:
        print("✅ GPU supports mixed precision training (Tensor Cores available)")
        mixed_precision_supported = True
    else:
        print("⚠️ GPU has limited mixed precision support")
        mixed_precision_supported = False
        
else:
    print("⚠️ No GPU available - using CPU")
    print("❌ FLUX.1-dev requires GPU acceleration for reasonable performance")
    flux_compatibility = "none"
    mixed_precision_supported = False

# Memory monitoring
try:
    memory_info = monitor_memory()
    print(f"\n💻 System Memory: {memory_info.get('system_total_gb', 0):.1f} GB total, {memory_info.get('system_percent', 0):.1f}% used")
    if 'gpu_total_gb' in memory_info:
        print(f"🎮 GPU Memory: {memory_info.get('gpu_total_gb', 0):.1f} GB total, {memory_info.get('gpu_percent', 0):.1f}% used")
except Exception as e:
    print(f"⚠️ Could not get memory info: {e}")

# Store hardware info for later use
hardware_info = {
    'gpu_available': torch.cuda.is_available(),
    'gpu_memory_gb': gpu_memory if torch.cuda.is_available() else 0,
    'flux_compatibility': flux_compatibility,
    'mixed_precision_supported': mixed_precision_supported
}

## 4. Create Directory Structure with Progress Tracking

In [ ]:
# Create necessary directories
directories_to_create = [
    data_dir / "real" / "images",
    data_dir / "processed",
    data_dir / "depth_maps",
    data_dir / "synthetic",
    output_dir,
    checkpoint_dir,
    Path("models"),
    Path("logs")
]

print("📁 Creating directory structure...")
progress_bar = create_progress_bar(len(directories_to_create), "Creating directories")

created_dirs = []
for directory in directories_to_create:
    try:
        directory.mkdir(parents=True, exist_ok=True)
        created_dirs.append(str(directory))
        progress_bar.update(1)
    except Exception as e:
        print(f"⚠️ Could not create directory {directory}: {e}")

progress_bar.close()
print(f"✅ Created {len(created_dirs)} directories")

# Display directory structure
print("\n📂 Directory Structure:")
for directory in created_dirs:
    print(f"  📁 {directory}")

## 5. Install Enhanced Dependencies for Depth-Anything-V2 and FLUX.1-dev

In [ ]:
import subprocess

# Define required packages for Depth-Anything-V2 and FLUX.1-dev
required_packages = [
    # Core PyTorch packages
    "torch>=2.0.0",
    "torchvision>=0.15.0",
    "torchaudio>=2.0.0",
    
    # Transformers and Diffusers for new models
    "transformers>=4.35.0",  # Updated for Depth-Anything-V2 support
    "diffusers>=0.24.0",     # Updated for FLUX.1-dev support
    "accelerate>=0.24.0",    # Enhanced acceleration for FLUX
    
    # FLUX-specific dependencies
    "sentencepiece>=0.1.99", # For FLUX text encoding
    "protobuf>=3.20.0",      # Protocol buffers for model serialization
    "safetensors>=0.4.0",    # Safe tensor loading for FLUX models
    
    # Depth-Anything-V2 dependencies
    "timm>=0.9.0",           # PyTorch Image Models for backbone
    "einops>=0.7.0",         # Tensor operations for depth models
    
    # Core image processing
    "opencv-python>=4.8.0",
    "Pillow>=10.0.0",
    "numpy>=1.24.0",
    
    # Utilities and monitoring
    "tqdm>=4.65.0",
    "psutil>=5.9.0",
    "matplotlib>=3.7.0",     # For visualization
    "scipy>=1.10.0",         # Scientific computing
    
    # Memory and performance optimization
    "xformers>=0.0.22",      # Memory-efficient attention for FLUX
    "bitsandbytes>=0.41.0"   # 8-bit optimization support
]

# Optional packages for enhanced performance
optional_packages = [
    "flash-attn>=2.3.0",     # Flash attention for even better memory efficiency
    "triton>=2.1.0"          # Triton kernels for optimized operations
]

def install_package(package):
    """Install a package using pip"""
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", package],
        capture_output=True,
        text=True
    )
    if result.returncode != 0:
        raise Exception(f"Installation failed: {result.stderr}")
    return result.stdout

print("📦 Installing required packages for Depth-Anything-V2 and FLUX.1-dev...")
progress_bar = create_progress_bar(len(required_packages), "Installing packages")

installed_packages = []
failed_packages = []

for package in required_packages:
    try:
        # Use retry logic for package installation
        retry_with_backoff(lambda: install_package(package), max_retries=2)
        installed_packages.append(package)
        progress_bar.update(1)
    except Exception as e:
        print(f"\n⚠️ Failed to install {package}: {e}")
        failed_packages.append(package)
        progress_bar.update(1)

progress_bar.close()

# Try to install optional packages
if hardware_info['gpu_available'] and hardware_info['flux_compatibility'] in ['optimal', 'adequate']:
    print("\n📦 Installing optional performance packages...")
    optional_progress = create_progress_bar(len(optional_packages), "Installing optional packages")
    
    for package in optional_packages:
        try:
            install_package(package)
            installed_packages.append(package)
            print(f"✅ Installed optional package: {package}")
        except Exception as e:
            print(f"⚠️ Optional package {package} failed: {e}")
        optional_progress.update(1)
    
    optional_progress.close()

# Display installation summary
installation_metrics = {
    'total_packages': len(required_packages),
    'installed_successfully': len([p for p in installed_packages if p in required_packages]),
    'failed_installations': len(failed_packages),
    'success_rate': len([p for p in installed_packages if p in required_packages]) / len(required_packages)
}

display_stage_summary("Package Installation", installation_metrics)

if failed_packages:
    print("\n⚠️ Failed packages:")
    for package in failed_packages:
        print(f"  - {package}")
    print("\n💡 Try installing failed packages manually or check your internet connection.")
    
    # Provide specific guidance for FLUX requirements
    if any('xformers' in pkg for pkg in failed_packages):
        print("\n🔧 XFormers installation tips:")
        print("  - Ensure you have the correct PyTorch version")
        print("  - Try: pip install xformers --index-url https://download.pytorch.org/whl/cu118")

## 6. Hardware-Adaptive Memory and Batch Size Configuration

In [ ]:
# Calculate optimal batch size based on available memory and FLUX requirements
print("🧠 Optimizing memory configuration for FLUX.1-dev...")

if batch_size == "auto":
    try:
        # Adjust max batch size based on FLUX compatibility
        if hardware_info['flux_compatibility'] == 'optimal':
            max_batch_size = 8
        elif hardware_info['flux_compatibility'] == 'adequate':
            max_batch_size = 4
        elif hardware_info['flux_compatibility'] == 'minimal':
            max_batch_size = 2
        else:
            max_batch_size = 1
            
        optimal_batch_size = get_optimal_batch_size(max_batch_size=max_batch_size)
        print(f"✅ Calculated optimal batch size for FLUX: {optimal_batch_size}")
        
        # Additional recommendations based on hardware
        if hardware_info['mixed_precision_supported']:
            print("✅ Mixed precision training recommended and supported")
            config['models']['mixed_precision'] = True
        else:
            print("⚠️ Mixed precision training not fully supported, using FP32")
            config['models']['mixed_precision'] = False
            
    except Exception as e:
        print(f"⚠️ Could not calculate optimal batch size: {e}")
        optimal_batch_size = 1  # Very conservative fallback for FLUX
        print(f"Using very conservative batch size for FLUX: {optimal_batch_size}")
else:
    optimal_batch_size = batch_size
    print(f"✅ Using configured batch size: {optimal_batch_size}")

# Update configuration with hardware-adaptive settings
try:
    config['processing']['batch_size'] = optimal_batch_size
    config['hardware'] = hardware_info
    
    # Add FLUX-specific optimizations
    config['optimizations'] = {
        'use_xformers': 'xformers>=0.0.22' in installed_packages,
        'use_flash_attention': 'flash-attn>=2.3.0' in installed_packages,
        'gradient_checkpointing': hardware_info['flux_compatibility'] in ['minimal', 'insufficient'],
        'cpu_offload': hardware_info['flux_compatibility'] == 'insufficient'
    }
    
    # Save updated configuration
    import json
    with open("config.json", "w") as f:
        json.dump(config, f, indent=2)
    
    print("✅ Configuration updated with hardware-adaptive settings")
    print(f"📊 Optimizations enabled:")
    for opt, enabled in config['optimizations'].items():
        status = "✅" if enabled else "❌"
        print(f"  {status} {opt}: {enabled}")
        
except Exception as e:
    print(f"⚠️ Could not save updated configuration: {e}")

# Display memory optimization suggestions
try:
    from utils.memory_optimizer import MemoryOptimizer
    optimizer = MemoryOptimizer()
    suggestions = optimizer.suggest_memory_optimizations()
    
    if suggestions:
        print("\n💡 Memory Optimization Suggestions:")
        for i, suggestion in enumerate(suggestions[:3], 1):
            print(f"  {i}. {suggestion}")
            
    # Add FLUX-specific suggestions
    print("\n🚀 FLUX.1-dev Specific Recommendations:")
    if hardware_info['flux_compatibility'] == 'insufficient':
        print("  ⚠️ Consider using CPU offloading for model components")
        print("  ⚠️ Use gradient checkpointing to reduce memory usage")
        print("  ⚠️ Process images one at a time (batch_size=1)")
    elif hardware_info['flux_compatibility'] == 'minimal':
        print("  💡 Enable gradient checkpointing for memory savings")
        print("  💡 Use mixed precision if supported")
        print("  💡 Consider smaller image resolutions initially")
    else:
        print("  ✅ Hardware is well-suited for FLUX.1-dev")
        print("  ✅ Can use standard batch sizes and full precision if needed")
        
except Exception as e:
    print(f"⚠️ Could not generate optimization suggestions: {e}")

## 7. Validate Configuration and Dependencies

In [ ]:
# Validate the updated configuration
print("🔍 Validating configuration and dependencies...")

try:
    # Validate configuration structure
    validation_result = validate_config(config)
    if validation_result['valid']:
        print("✅ Configuration validation passed")
    else:
        print(f"⚠️ Configuration validation issues: {validation_result['errors']}")
        
except Exception as e:
    print(f"⚠️ Could not validate configuration: {e}")

# Test critical imports for new models
critical_imports = {
    'torch': 'PyTorch',
    'transformers': 'Transformers (for Depth-Anything-V2)',
    'diffusers': 'Diffusers (for FLUX.1-dev)',
    'timm': 'TIMM (for Depth-Anything-V2 backbone)',
    'einops': 'Einops (for tensor operations)'
}

print("\n🧪 Testing critical imports:")
import_results = {}

for module, description in critical_imports.items():
    try:
        __import__(module)
        print(f"✅ {description}: Available")
        import_results[module] = True
    except ImportError as e:
        print(f"❌ {description}: Failed - {e}")
        import_results[module] = False

# Test optional performance imports
optional_imports = {
    'xformers': 'XFormers (memory optimization)',
    'bitsandbytes': 'BitsAndBytes (8-bit optimization)'
}

print("\n🔧 Testing optional performance imports:")
for module, description in optional_imports.items():
    try:
        __import__(module)
        print(f"✅ {description}: Available")
        import_results[module] = True
    except ImportError as e:
        print(f"⚠️ {description}: Not available - {e}")
        import_results[module] = False

# Check model accessibility (without downloading)
print("\n🤖 Checking model accessibility:")
model_checks = {
    'depth_model': config['models']['depth_model'],
    'base_model': config['models']['base_model'],
    'controlnet_model': config['models']['controlnet_model']
}

for model_type, model_name in model_checks.items():
    try:
        # Just check if we can create the model config (doesn't download)
        if 'depth-anything' in model_name.lower():
            print(f"✅ {model_type}: {model_name} (Depth-Anything-V2 format recognized)")
        elif 'flux' in model_name.lower():
            print(f"✅ {model_type}: {model_name} (FLUX format recognized)")
        else:
            print(f"⚠️ {model_type}: {model_name} (format not specifically recognized)")
    except Exception as e:
        print(f"❌ {model_type}: {model_name} - {e}")

## 8. Save Setup Results and Environment Information

In [ ]:
# Collect comprehensive environment information
environment_info = {
    'platform': platform.system(),
    'python_version': sys.version.split()[0],
    'torch_version': torch.__version__,
    'cuda_available': torch.cuda.is_available(),
    'gpu_count': torch.cuda.device_count() if torch.cuda.is_available() else 0,
    'optimal_batch_size': optimal_batch_size,
    'directories_created': created_dirs,
    'installed_packages': installed_packages,
    'failed_packages': failed_packages,
    'import_results': import_results,
    'hardware_info': hardware_info
}

if torch.cuda.is_available():
    environment_info.update({
        'gpu_name': torch.cuda.get_device_name(0),
        'gpu_memory_gb': torch.cuda.get_device_properties(0).total_memory / (1024**3),
        'cuda_version': torch.version.cuda
    })

# Save setup results
setup_results = {
    'environment_info': environment_info,
    'configuration': config,
    'setup_status': 'completed',
    'installation_metrics': installation_metrics,
    'model_compatibility': {
        'depth_anything_v2_ready': import_results.get('timm', False) and import_results.get('einops', False),
        'flux_ready': import_results.get('diffusers', False) and hardware_info['flux_compatibility'] != 'none',
        'performance_optimized': import_results.get('xformers', False) or import_results.get('flash-attn', False)
    }
}

setup_metadata = {
    'setup_time': '2024-01-12T00:00:00Z',  # This would be actual timestamp
    'notebook_version': '2.0_enhanced_flux_depth_anything_v2',
    'utilities_version': '1.0',
    'models_supported': ['Depth-Anything-V2-Large', 'FLUX.1-dev', 'FLUX-ControlNet-Depth']
}

try:
    success = save_stage_results(setup_results, "setup", setup_metadata)
    if success:
        print("✅ Setup results saved successfully")
        print(f"📁 Results saved to: {output_dir / 'setup'}")
    else:
        print("⚠️ Could not save setup results")
except Exception as e:
    print(f"⚠️ Error saving setup results: {e}")

# Display final setup summary
print("\n🎉 Enhanced Setup and Installation Completed!")
print(f"\n📊 Setup Summary:")
print(f"  🤖 Models: Depth-Anything-V2 + FLUX.1-dev ready")
print(f"  💾 GPU Memory: {hardware_info.get('gpu_memory_gb', 0):.1f} GB ({hardware_info['flux_compatibility']} for FLUX)")
print(f"  🔧 Batch Size: {optimal_batch_size}")
print(f"  ⚡ Mixed Precision: {config['models']['mixed_precision']}")
print(f"  📦 Packages: {installation_metrics['installed_successfully']}/{installation_metrics['total_packages']} installed")

print("\n📋 Next Steps:")
print("1. 📂 Add your real beach images to the data/real/images/ directory")
print("2. ▶️ Run notebook 02: Data Import and Preprocessing")
print("3. 🔄 Continue with notebook 03: Depth-Anything-V2 Extraction")
print("4. 🎨 Proceed to notebook 05: FLUX ControlNet Synthetic Generation")

if failed_packages:
    print("\n⚠️ Note: Some packages failed to install. The pipeline may still work, but performance could be affected.")
    
if hardware_info['flux_compatibility'] in ['minimal', 'insufficient']:
    print("\n💡 Hardware Recommendation: Consider upgrading GPU memory for optimal FLUX.1-dev performance.")

## 9. Memory Cleanup

In [ ]:
# Clean up large variables to free memory
large_variables = [
    'setup_results',
    'environment_info',
    'installation_metrics'
]

try:
    cleanup_variables(large_variables)
    print("✅ Memory cleanup completed")
    
    # Show final memory status
    final_memory = monitor_memory()
    print(f"💻 Final system memory usage: {final_memory.get('system_percent', 0):.1f}%")
    if 'gpu_percent' in final_memory:
        print(f"🎮 Final GPU memory usage: {final_memory.get('gpu_percent', 0):.1f}%")
        
except Exception as e:
    print(f"⚠️ Memory cleanup warning: {e}")

print("\n🚀 Ready for Depth-Anything-V2 and FLUX.1-dev pipeline!")